In [3]:
#load_ext autotime
import pandas as pd
import geopandas as gpd
import geopy
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
import matplotlib.pyplot as plt
import plotly_express as px
import tqdm
from tqdm.notebook import tqdm_notebook
import os
import numpy as np
import glob

In [86]:
locator = Nominatim(user_agent="measurements")

In [85]:
inpath = r"T:\DCProjects\StoryMap\BikeCounting\BikeShare\Data\Trips"

In [115]:
file = "trips_peace_health_rides_06_01_2018-06_30_2018.csv"

In [87]:
odf = pd.read_csv(os.path.join(inpath, file))
odf.head()

,User ID,Route ID,Bike Region ID,Payment Plan,Start Hub,Start Area,Start Special Area,Start Latitude,Start Longitude,Start Date,...,Bike ID,Bike Name,Distance [Miles],Duration,Rental Access Path,Multiple Rental,Ride cost,Fees,Bonuses,Total cost
0,787416,7269180,NaN,Single Ride,"16th & Hilyard, West side",Phase 1 System Area,NaN,44.04109,-123.081935,2018-06-01,...,19650,0120,0.30,00:03:49,keypad_phone_number,False,0.0,0.0,0.0,0.0
1,856344,7269203,NaN,UO Monthly Plan,Matthew Knight Arena,Phase 1 System Area,NaN,44.045478333333335,-123.06537166666666,2018-06-01,...,19624,0094,0.71,00:08:44,keypad,False,0.0,0.0,0.0,0.0
2,856729,7269205,NaN,UO Monthly Plan,Matthew Knight Arena,Phase 1 System Area,NaN,44.045471666666664,-123.06526666666667,2018-06-01,...,19606,0075,0.69,00:08:23,keypad,False,0.0,0.0,0.0,0.0
3,781643,7269215,NaN,UO Monthly Plan,Matthew Knight Arena,Phase 1 System Area,NaN,44.045375,-123.06513666666666,2018-06-01,...,19798,0268,1.84,01:41:17,keypad_phone_number,False,0.0,0.0,0.0,0.0
4,777445,7269218,NaN,UO Monthly Plan,10th & Olive SE,Phase 1 System Area,NaN,44.049508333333335,-123.094235,2018-06-01,...,19418,0017,1.38,00:09:33,keypad,False,0.0,0.0,0.0,0.0


In [88]:
odf.columns

Index(['User ID', 'Route ID', 'Bike Region ID', 'Payment Plan', 'Start Hub',
       'Start Area', 'Start Special Area', 'Start Latitude', 'Start Longitude',
       'Start Date', 'Start Time', 'End Hub', 'End Area', 'End Special Area',
       'End Latitude', 'End Longitude', 'End Date', 'End Time', 'Member Type',
       'Trip Type', 'Bike ID', 'Bike Name', 'Distance [Miles]', 'Duration',
       'Rental Access Path', 'Multiple Rental', 'Ride cost', 'Fees', 'Bonuses',
       'Total cost'],
      dtype='object')

In [89]:
df = odf 

In [90]:
df = df[(df['Start Latitude'] != " - ") | (df['End Latitude'] != " - " ) | (df['Start Longitude'] != " - ") | (df['End Longitude'] != " - ")]

In [91]:
df["startgeom"] = df["Start Latitude"].map(str) + ',' + df['Start Longitude'].map(str)
df["startgeom"][0]

<ipython-input-91-3ae606a348da>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["startgeom"] = df["Start Latitude"].map(str) + ',' + df['Start Longitude'].map(str)


'44.04109,-123.081935'

In [92]:
df["endgeom"] = df["End Latitude"].map(str) + ',' + df['End Longitude'].map(str)
df["endgeom"][0]

<ipython-input-92-d7f1bfc5249d>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["endgeom"] = df["End Latitude"].map(str) + ',' + df['End Longitude'].map(str)


'44.03763333333333,-123.08355333333333'

In [93]:
locator = Nominatim(user_agent="measurements", timeout=10)
rgeocode = RateLimiter(locator.reverse, min_delay_seconds=0.00001)

In [94]:
tqdm_notebook.pandas()

In [96]:
df = df[df["Route ID"] != 7763775]

In [97]:
df['startaddress'] = df['startgeom'].progress_apply(rgeocode)
df.head()

  0%|          | 0/23858 [00:00<?, ?it/s]

,User ID,Route ID,Bike Region ID,Payment Plan,Start Hub,Start Area,Start Special Area,Start Latitude,Start Longitude,Start Date,...,Duration,Rental Access Path,Multiple Rental,Ride cost,Fees,Bonuses,Total cost,startgeom,endgeom,startaddress
0,787416,7269180,NaN,Single Ride,"16th & Hilyard, West side",Phase 1 System Area,NaN,44.04109,-123.081935,2018-06-01,...,00:03:49,keypad_phone_number,False,0.0,0.0,0.0,0.0,"44.04109,-123.081935","44.03763333333333,-123.08355333333333","(Peace Lutheran, East 17th Avenue, College Hil..."
1,856344,7269203,NaN,UO Monthly Plan,Matthew Knight Arena,Phase 1 System Area,NaN,44.045478333333335,-123.06537166666666,2018-06-01,...,00:08:44,keypad,False,0.0,0.0,0.0,0.0,"44.045478333333335,-123.06537166666666","44.04586833333333,-123.07854666666667","(Matthew Knight Arena, Franklin Boulevard, Eug..."
2,856729,7269205,NaN,UO Monthly Plan,Matthew Knight Arena,Phase 1 System Area,NaN,44.045471666666664,-123.06526666666667,2018-06-01,...,00:08:23,keypad,False,0.0,0.0,0.0,0.0,"44.045471666666664,-123.06526666666667","44.045883333333336,-123.07848833333334","(Matthew Knight Arena, Franklin Boulevard, Eug..."
3,781643,7269215,NaN,UO Monthly Plan,Matthew Knight Arena,Phase 1 System Area,NaN,44.045375,-123.06513666666666,2018-06-01,...,01:41:17,keypad_phone_number,False,0.0,0.0,0.0,0.0,"44.045375,-123.06513666666666","44.041065,-123.08195833333333","(Matthew Knight Arena, Franklin Boulevard, Eug..."
4,777445,7269218,NaN,UO Monthly Plan,10th & Olive SE,Phase 1 System Area,NaN,44.049508333333335,-123.094235,2018-06-01,...,00:09:33,keypad,False,0.0,0.0,0.0,0.0,"44.049508333333335,-123.094235","44.04582333333333,-123.07851","(Luckey's, Olive Street, Eugene, Lane County, ..."


In [98]:
df['endaddress'] = df['endgeom'].progress_apply(rgeocode)
df.head()

  0%|          | 0/23858 [00:00<?, ?it/s]

,User ID,Route ID,Bike Region ID,Payment Plan,Start Hub,Start Area,Start Special Area,Start Latitude,Start Longitude,Start Date,...,Rental Access Path,Multiple Rental,Ride cost,Fees,Bonuses,Total cost,startgeom,endgeom,startaddress,endaddress
0,787416,7269180,NaN,Single Ride,"16th & Hilyard, West side",Phase 1 System Area,NaN,44.04109,-123.081935,2018-06-01,...,keypad_phone_number,False,0.0,0.0,0.0,0.0,"44.04109,-123.081935","44.03763333333333,-123.08355333333333","(Peace Lutheran, East 17th Avenue, College Hil...","(Eugene Family YMCA, 2055, Patterson Street, C..."
1,856344,7269203,NaN,UO Monthly Plan,Matthew Knight Arena,Phase 1 System Area,NaN,44.045478333333335,-123.06537166666666,2018-06-01,...,keypad,False,0.0,0.0,0.0,0.0,"44.045478333333335,-123.06537166666666","44.04586833333333,-123.07854666666667","(Matthew Knight Arena, Franklin Boulevard, Eug...","(University of Oregon Station - Bay B, Univers..."
2,856729,7269205,NaN,UO Monthly Plan,Matthew Knight Arena,Phase 1 System Area,NaN,44.045471666666664,-123.06526666666667,2018-06-01,...,keypad,False,0.0,0.0,0.0,0.0,"44.045471666666664,-123.06526666666667","44.045883333333336,-123.07848833333334","(Matthew Knight Arena, Franklin Boulevard, Eug...","(Chiles Hall, University Station North - Bay C..."
3,781643,7269215,NaN,UO Monthly Plan,Matthew Knight Arena,Phase 1 System Area,NaN,44.045375,-123.06513666666666,2018-06-01,...,keypad_phone_number,False,0.0,0.0,0.0,0.0,"44.045375,-123.06513666666666","44.041065,-123.08195833333333","(Matthew Knight Arena, Franklin Boulevard, Eug...","(Peace Lutheran, East 17th Avenue, College Hil..."
4,777445,7269218,NaN,UO Monthly Plan,10th & Olive SE,Phase 1 System Area,NaN,44.049508333333335,-123.094235,2018-06-01,...,keypad,False,0.0,0.0,0.0,0.0,"44.049508333333335,-123.094235","44.04582333333333,-123.07851","(Luckey's, Olive Street, Eugene, Lane County, ...","(Chiles Hall, University Station North - Bay C..."


In [99]:
def getLoc(x):
    if x is not None:
        loc = x.raw['address']
        a = [key for key, value in loc.items()][0]
        b = [key for key, value in loc.items()][1]
        if  a == 'house_number':
            res = loc['road'] + ' ' + loc['house_number']
        else:
            res = loc[a]
            if res == "University of Oregon":
                res = "UO " + loc[b]
    else:
        res = ""
    return res

In [100]:
df['startname'] = df.startaddress.apply(lambda x: getLoc(x))

In [101]:
df['endname'] = df.endaddress.apply(lambda x: getLoc(x))

In [102]:
df.columns

Index(['User ID', 'Route ID', 'Bike Region ID', 'Payment Plan', 'Start Hub',
       'Start Area', 'Start Special Area', 'Start Latitude', 'Start Longitude',
       'Start Date', 'Start Time', 'End Hub', 'End Area', 'End Special Area',
       'End Latitude', 'End Longitude', 'End Date', 'End Time', 'Member Type',
       'Trip Type', 'Bike ID', 'Bike Name', 'Distance [Miles]', 'Duration',
       'Rental Access Path', 'Multiple Rental', 'Ride cost', 'Fees', 'Bonuses',
       'Total cost', 'startgeom', 'endgeom', 'startaddress', 'endaddress',
       'startname', 'endname'],
      dtype='object')

In [103]:
df.loc[df['Start Hub'].isnull(), 'Start Hub'] = df.loc[df['Start Hub'].isnull(), 'startname']

In [104]:
df.loc[df['End Hub'].isnull(), 'End Hub'] = df.loc[df['End Hub'].isnull(), 'endname']

In [105]:
path = r"T:\DCProjects\StoryMap\BikeCounting\BikeShare\Data\Output\review"

In [108]:
df.to_csv(os.path.join(path, file), index=False)

In [116]:
df.head()

,User ID,Route ID,Bike Region ID,Payment Plan,Start Hub,Start Area,Start Special Area,Start Latitude,Start Longitude,Start Date,...,Ride cost,Fees,Bonuses,Total cost,startgeom,endgeom,startaddress,endaddress,startname,endname
0,787416,7269180,NaN,Single Ride,"16th & Hilyard, West side",Phase 1 System Area,NaN,44.04109,-123.081935,2018-06-01,...,0.0,0.0,0.0,0.0,"44.04109,-123.081935","44.03763333333333,-123.08355333333333","(Peace Lutheran, East 17th Avenue, College Hil...","(Eugene Family YMCA, 2055, Patterson Street, C...",Peace Lutheran,Eugene Family YMCA
1,856344,7269203,NaN,UO Monthly Plan,Matthew Knight Arena,Phase 1 System Area,NaN,44.045478333333335,-123.06537166666666,2018-06-01,...,0.0,0.0,0.0,0.0,"44.045478333333335,-123.06537166666666","44.04586833333333,-123.07854666666667","(Matthew Knight Arena, Franklin Boulevard, Eug...","(University of Oregon Station - Bay B, Univers...",Matthew Knight Arena,University of Oregon Station - Bay B
2,856729,7269205,NaN,UO Monthly Plan,Matthew Knight Arena,Phase 1 System Area,NaN,44.045471666666664,-123.06526666666667,2018-06-01,...,0.0,0.0,0.0,0.0,"44.045471666666664,-123.06526666666667","44.045883333333336,-123.07848833333334","(Matthew Knight Arena, Franklin Boulevard, Eug...","(Chiles Hall, University Station North - Bay C...",Matthew Knight Arena,Chiles Hall
3,781643,7269215,NaN,UO Monthly Plan,Matthew Knight Arena,Phase 1 System Area,NaN,44.045375,-123.06513666666666,2018-06-01,...,0.0,0.0,0.0,0.0,"44.045375,-123.06513666666666","44.041065,-123.08195833333333","(Matthew Knight Arena, Franklin Boulevard, Eug...","(Peace Lutheran, East 17th Avenue, College Hil...",Matthew Knight Arena,Peace Lutheran
4,777445,7269218,NaN,UO Monthly Plan,10th & Olive SE,Phase 1 System Area,NaN,44.049508333333335,-123.094235,2018-06-01,...,0.0,0.0,0.0,0.0,"44.049508333333335,-123.094235","44.04582333333333,-123.07851","(Luckey's, Olive Street, Eugene, Lane County, ...","(Chiles Hall, University Station North - Bay C...",Luckey's,Chiles Hall


In [117]:
ndf = df.drop(['startgeom', 'endgeom', 'startaddress', 'endaddress',
       'startname', 'endname'], axis=1)

In [118]:
outpath = r"T:\DCProjects\StoryMap\BikeCounting\BikeShare\Data\Output"

In [119]:
ndf.to_csv(os.path.join(outpath, file), index=False)

In [5]:
files = glob.glob(os.path.join(path, "*.csv"))

In [73]:
def review_hubs(re_df, strp='Eugene'):
    for hub in ["Start", "End"]:
        if hub+' Hub' in re_df.columns:
            re = re_df[re_df[hub+' Hub'].str.contains(strp, na=False)][[hub+' Latitude', hub+' Longitude']]
        else:
            re = re_df[re_df[hub+'.Hub'].str.contains(strp, na=False)][[hub+'.Latitude', hub+'.Longitude']]
        
        if re.shape[0] > 0:
            re.columns = ['Lat', 'Lon']
            re["geom"] = re["Lat"].map(str) + ',' + re['Lon'].map(str)
            locator = Nominatim(user_agent="measurements", timeout=10)
            rgeocode = RateLimiter(locator.reverse, min_delay_seconds=0.00001)
            tqdm_notebook.pandas()
            re['address'] = re['geom'].progress_apply(rgeocode)
            re['name'] =re.address.apply(getLoc)
            if hub+' Hub' in re_df.columns:
                re_df.loc[re_df[hub+' Hub'].str.contains(strp, na=False), hub+' Hub'] = re.name.values  
            else:
                re_df.loc[re_df[hub+'.Hub'].str.contains(strp, na=False), hub+'.Hub'] = re.name.values
    return re_df

In [111]:
def review_hubs(re_df):
    for hub in ["Start", "End"]:
        if hub+' Hub' in re_df.columns:
            if re_df[re_df[hub+' Hub'].isnull()].shape[0] > 0:
                re = re_df[re_df[hub+' Hub'].isnull()][[hub+' Latitude', hub+' Longitude']]
            else:
                re = re_df[re_df[hub+' Hub'].str.isnumeric()][[hub+' Latitude', hub+' Longitude']]
        else:
            re = re_df[re_df[hub+'.Hub'].str.isnumeric()][[hub+'.Latitude', hub+'.Longitude']]
        
        if re.shape[0] > 0:
            re.columns = ['Lat', 'Lon']
            re["geom"] = re["Lat"].map(str) + ',' + re['Lon'].map(str)
            locator = Nominatim(user_agent="measurements", timeout=10)
            rgeocode = RateLimiter(locator.reverse, min_delay_seconds=0.00001)
            tqdm_notebook.pandas()
            re['address'] = re['geom'].progress_apply(rgeocode)
            re['name'] =re.address.apply(getLoc)
            if hub+' Hub' in re_df.columns:
                if re_df[re_df[hub+' Hub'].isnull()].shape[0] > 0:
                    re_df.loc[re_df[hub+' Hub'].isnull(), hub+' Hub'] = re.name.values
                else:
                    re_df.loc[re_df[hub+' Hub'].str.isnumeric(), hub+' Hub'] = re.name.values  
            else:
                re_df.loc[re_df[hub+'.Hub'].str.isnumeric(), hub+'.Hub'] = re.name.values
    return re_df

In [82]:
for file in files:
    print(file)
    re_df = pd.read_csv(file)
    #re_df = review_hubs(re_df)
    re_df = review_hubs(re_df, strp='Eugene-Springfield')
    re_df.to_csv(file, index=False)

T:\DCProjects\StoryMap\BikeCounting\BikeShare\Data\Output\review\trips_peace_health_rides_01_01_2019-01_31_2019.csv


  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

T:\DCProjects\StoryMap\BikeCounting\BikeShare\Data\Output\review\trips_peace_health_rides_01_01_2020-01_31_2020.csv


  0%|          | 0/1 [00:00<?, ?it/s]

...

  0%|          | 0/1 [00:00<?, ?it/s]

T:\DCProjects\StoryMap\BikeCounting\BikeShare\Data\Output\review\trips_peace_health_rides_12_01_2019-12_31_2019.csv


  0%|          | 0/1 [00:00<?, ?it/s]

T:\DCProjects\StoryMap\BikeCounting\BikeShare\Data\Output\review\trips_peace_health_rides_12_01_2020-12_31_2020.csv
T:\DCProjects\StoryMap\BikeCounting\BikeShare\Data\Output\review\trips_peace_health_rides_12_01_2021-12_31_2021.csv


  0%|          | 0/2 [00:00<?, ?it/s]

In [109]:
file = r"T:\DCProjects\StoryMap\BikeCounting\BikeShare\Data\Output\review\trips_peace_health_rides_12_01_2021-12_31_2021.csv"
file

'T:\\DCProjects\\StoryMap\\BikeCounting\\BikeShare\\Data\\Output\\review\\trips_peace_health_rides_12_01_2021-12_31_2021.csv'

In [110]:
re_df = pd.read_csv(file)

In [62]:
re_df.columns

Index(['User.ID', 'Route.ID', 'Start.Hub', 'Start.Latitude', 'Start.Longitude',
       'Start.Date', 'Start.Time', 'End.Hub', 'End.Latitude', 'End.Longitude',
       'End.Date', 'End.Time', 'Bike.ID', 'Bike.Name', 'Distance..Miles.',
       'Duration'],
      dtype='object')

In [55]:
re_df.head()

,User ID,Route ID,Bike Region ID,Payment Plan,Start Hub,Start Area,Start Special Area,Start Latitude,Start Longitude,Start Date,...,Bike ID,Bike Name,Distance [Miles],Duration,Rental Access Path,Multiple Rental,Ride cost,Fees,Bonuses,Total cost
0,766963,12817400,NaN,UO Monthly Plan,Uniquely Chengdu,Phase 1 System Area,NaN,44.045435,-123.080060,2019-12-01,...,19591,0057,0.76,00:07:59,keypad,False,0.0,0.0,0.0,0.0
1,1702290,12817409,NaN,UO Monthly Plan,Thompson-Roach Building,Phase 1 System Area,NaN,44.045438,-123.084452,2019-12-01,...,19703,0173,0.35,00:13:51,keypad,True,0.0,0.0,0.0,0.0
2,1665396,12817412,NaN,Monthly Plan,Monroe St & Blair Blvd,Phase 1 System Area,NaN,44.051473,-123.105138,2019-12-01,...,19603,0072,1.19,00:07:58,keypad,False,0.0,0.0,0.0,0.0
3,1734280,12817414,NaN,UO Monthly Plan,Thompson-Roach Building,Phase 1 System Area,NaN,44.045413,-123.084448,2019-12-01,...,19788,0258,0.43,00:03:13,keypad,False,0.0,0.0,0.0,0.0
4,1702290,12817415,NaN,UO Monthly Plan,Thompson-Roach Building,Phase 1 System Area,NaN,44.045507,-123.084462,2019-12-01,...,19661,0131,0.05,00:06:08,keypad,True,0.0,0.0,0.0,0.0


In [113]:
file

'T:\\DCProjects\\StoryMap\\BikeCounting\\BikeShare\\Data\\Output\\review\\trips_peace_health_rides_12_01_2021-12_31_2021.csv'

In [112]:
re_df = review_hubs(re_df)

In [114]:
re_df.to_csv(os.path.join(path, file), index=False)

In [64]:
hub = "Start"

In [75]:
strp = "Eugene-Springfield"

In [76]:
re = re_df[re_df[hub+' Hub'].str.contains(strp, na=False)]

In [78]:
re.head()

,User ID,Route ID,Bike Region ID,Payment Plan,Start Hub,Start Area,Start Special Area,Start Latitude,Start Longitude,Start Date,...,Bike ID,Bike Name,Distance [Miles],Duration,Rental Access Path,Multiple Rental,Ride cost,Fees,Bonuses,Total cost
3345,2207423,14202390,NaN,Monthly Plan,Eugene-Springfield Highway #227,Phase 1 System Area,NaN,44.051752,-123.101705,2021-12-11,...,19581,0050,1.42,00:10:14,mobile,False,0.0,0.0,0.0,0.0
3814,2207423,14205445,NaN,Monthly Plan,Eugene-Springfield Highway #227,Phase 1 System Area,NaN,44.051790,-123.101713,2021-12-17,...,19767,0237,1.41,00:11:39,mobile,False,0.0,0.0,0.0,0.0


In [79]:
testdf = review_hubs(re_df, strp = strp)

  0%|          | 0/2 [00:00<?, ?it/s]

In [80]:
testdf[testdf[hub+' Hub'].str.contains(strp, na=False)] 

,User ID,Route ID,Bike Region ID,Payment Plan,Start Hub,Start Area,Start Special Area,Start Latitude,Start Longitude,Start Date,...,Bike ID,Bike Name,Distance [Miles],Duration,Rental Access Path,Multiple Rental,Ride cost,Fees,Bonuses,Total cost
3345,2207423,14202390,NaN,Monthly Plan,Eugene-Springfield Highway #227,Phase 1 System Area,NaN,44.051752,-123.101705,2021-12-11,...,19581,0050,1.42,00:10:14,mobile,False,0.0,0.0,0.0,0.0
3814,2207423,14205445,NaN,Monthly Plan,Eugene-Springfield Highway #227,Phase 1 System Area,NaN,44.051790,-123.101713,2021-12-17,...,19767,0237,1.41,00:11:39,mobile,False,0.0,0.0,0.0,0.0


In [71]:
testdf.loc[8631, :]

User.ID                              1170002
Route.ID                            10670952
Start.Hub           South Eugene High School
Start.Latitude                        44.036
Start.Longitude                  -123.085573
Start.Date                        2019-01-23
Start.Time                             14:20
End.Hub                    15th & University
End.Latitude                       44.043122
End.Longitude                    -123.074752
End.Date                          2019-01-23
End.Time                               14:28
Bike.ID                                27634
Bike.Name                               0313
Distance..Miles.                        0.99
Duration                            00:07:25
Name: 8631, dtype: object

In [67]:
re_df[re_df[hub+'.Hub'].str.contains(strp, na=False)].head

<bound method NDFrame.head of       User.ID  Route.ID     Start.Hub  Start.Latitude  Start.Longitude  \
7625  1170002  10583547  Unit 149-156       44.035897      -123.085982   
8631  1170002  10670952  Unit 165-172       44.036000      -123.085573   
8862  1416265  10684312  Unit 225-232       44.035232      -123.084248   
9089  1170002  10702778  Unit 149-156       44.035895      -123.085792   

      Start.Date Start.Time             End.Hub  End.Latitude  End.Longitude  \
7625  2019-01-07      13:21   15th & University     44.043068    -123.074728   
8631  2019-01-23      14:20   15th & University     44.043122    -123.074752   
8862  2019-01-25      16:53  Erb Memorial Union     44.045380    -123.073143   
9089  2019-01-29      09:36      13th & Kincaid     44.045898    -123.078493   

        End.Date End.Time  Bike.ID Bike.Name  Distance..Miles.  Duration  
7625  2019-01-07    13:32    19849     0165               1.34  00:10:47  
8631  2019-01-23    14:28    27634      0313    

In [47]:
re = re_df[re_df[hub+' Hub'].str.contains(strp, na=False)][[hub+' Latitude', hub+' Longitude']]

In [84]:
geolocator = Nominatim(user_agent="measurements")
location = geolocator.geocode("Eugene-Springfield")
address, (latitude, longitude) = location
address

'Eugene-Springfield, Willamette Street, Eugene, Lane County, Oregon, 97401, United States'

In [30]:
re.columns = ['Lat', 'Lon']
re["geom"] = re["Lat"].map(str) + ',' + re['Lon'].map(str)
locator = Nominatim(user_agent="measurements", timeout=10)
rgeocode = RateLimiter(locator.reverse, min_delay_seconds=0.00001)
tqdm_notebook.pandas()
re['address'] = re['geom'].progress_apply(rgeocode)
re['name'] =re.address.apply(getLoc)

  0%|          | 0/1 [00:00<?, ?it/s]

In [31]:
re.name.values

array(['76'], dtype=object)

In [32]:
re.address.values

array([Location(76, West 18th Avenue, Westmoreland Commercial, Eugene, Lane County, Oregon, 97405, United States, (44.0406689, -123.11734339775421, 0.0))],
      dtype=object)

In [35]:
re.address.values[0].raw

{'place_id': 162223077,
 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright',
 'osm_type': 'way',
 'osm_id': 244974435,
 'lat': '44.0406689',
 'lon': '-123.11734339775421',
 'display_name': '76, West 18th Avenue, Westmoreland Commercial, Eugene, Lane County, Oregon, 97405, United States',
 'address': {'amenity': '76',
  'road': 'West 18th Avenue',
  'commercial': 'Westmoreland Commercial',
  'city': 'Eugene',
  'county': 'Lane County',
  'state': 'Oregon',
  'ISO3166-2-lvl4': 'US-OR',
  'postcode': '97405',
  'country': 'United States',
  'country_code': 'us'},
 'boundingbox': ['44.0406071', '44.040741', '-123.1175184', '-123.1171684']}